http://www.nuforc.org/webreports/ndxe202004.html

https://data.world/timothyrenner/ufo-sightings

In [61]:
from bs4 import BeautifulSoup as bsoup
from urllib.request import Request, urlopen
import csv
import requests
import pandas as pd
import numpy as np

In [92]:
def get_page(url):
    #Make the request and open the url :
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()

    #Create a beutiful soup object so we can work with the data
    soup = bsoup(webpage, 'html.parser')

    #Sanity check 
    return soup

In [93]:
def get_iterable_data(table):
    for c in table:
        yield c.getText()

In [94]:
def get_sighting_detail(url_string):
    detail_page = get_page(url_string)
    detail_page = detail_page.find("tbody")
    rows = detail_page.find_all("tr")
    d = get_iterable_data(rows)
    next(d)
    return next(d)

In [128]:
def monthly_report_detail(url_string, col_names, sightings, remaining):
    # func_sightings = pd.DataFrame(columns = col_names)
    soup = get_page(url_string)
    temp = dict()

    results = soup.find_all('tr', attrs={'valign': 'TOP'})
    index = len(sightings)
    count = 0
    for result in results:
        if remaining > 0:
            remaining -= 1
            count += 1
            continue
        # print(f"Length: {len(sightings)}, index: {index}, count: {count}, remaining: {remaining}")
        table_cols = result.find_all('td')    
        col_text = get_iterable_data(table_cols)
        temp["Date_Time"] = next(col_text)
        temp["City"] = next(col_text)
        temp["State"] = next(col_text)
        temp["Shape"] = next(col_text)
        temp["Duration"] = next(col_text)
        temp["Summary"] = next(col_text)
        temp["Posted"] = next(col_text)
        temp["Detail_Link"] = "http://www.nuforc.org/webreports/" + result.find("a").get("href")
        temp["Detail_Summary"] = get_sighting_detail(temp["Detail_Link"])

        # func_sightings = func_sightings.append(temp, ignore_index = True)
        sightings.loc[index] = temp
        index += 1
    return len(results)

In [113]:
col_names = ["Date_Time", "City", "State", "Shape", "Duration", "Summary", "Posted", "Detail_Link", "Detail_Summary"]
# sightings = pd.DataFrame(columns = col_names)

In [ ]:
url_string = "http://www.nuforc.org/webreports/ndxevent.html"
monthly_report = get_page(url_string)
monthly_report_list = monthly_report.find_all('tr', attrs={'valign': 'TOP'})
# print(monthly_report_list)

remaining = len(sightings)
for report in monthly_report_list:
    cols = report.find_all("td")
    report_number = int(cols[1].getText())
    
    if remaining > report_number:
        remaining = remaining - report_number
        continue
    
    if remaining > 0:
        print(f"Resuming: {len(sightings)}")
    
    monthly_report_url = "http://www.nuforc.org/webreports/" + report.find("a").get("href")
    
    sightings_added = monthly_report_detail(monthly_report_url, col_names, sightings, remaining)
    print(f"Just added {sightings_added} sightings, total now {len(sightings)}")
    
    remaining = 0

Resuming: 2447
Just added 454 sightings, total now 2792


In [122]:
len(sightings)

2431

In [112]:
sightings

,Date_Time,City,State,Shape,Duration,Summary,Posted,Detail_Link,Detail_Summary
0,4/23/21 06:30,Blackshear,GA,Circle,9 minutes,Very strange ((NUFORC Note: Rocket launch f...,4/23/21,http://www.nuforc.org/webreports/162/S162815.html,\nVery strangeI have recorded a video of this ...
1,4/23/21 06:00,Mechanicsville,VA,Circle,Seconds,Ball in the sky ((NUFORC Note: Rocket launc...,4/23/21,http://www.nuforc.org/webreports/162/S162814.html,\nBall in the skyObject appears as a white bal...
2,4/23/21 06:00,Vero Beach,FL,Light,5 minutes,I was driving and saw something strange in the...,4/23/21,http://www.nuforc.org/webreports/162/S162822.html,\nI was driving and saw something strange in t...
